In [1]:
using DrWatson
@quickactivate "ad_lib"
using MLDatasets: MNIST
using Printf
using Statistics
import Base: -
include(srcdir("operators.jl"))


backward (generic function with 14 methods)

In [2]:
function dense(w, b, x, activation)
    return activation(w * x .+ b)
end

function dense(w, x, activation)
    return activation(w * x)
end

function dense(w, x)
    return w * x
end

function crossentropy(y, actual_class)
    select(.-log.(y), actual_class)
end

function relu(x)
    max.(x, Constant(0))
end

-(x::Vector, y::Matrix) = vec(x .- y)

function update_var!(x::Variable, alpha)
    x.output = x.output - (x.gradient * alpha)
end

update_var! (generic function with 1 method)

In [3]:
NUM_OF_CLASSES = 10
LEARNING_RATE = 0.001
EPOCHS = 3


# variables that will be modified
b = Variable(rand(Float64, NUM_OF_CLASSES), name="dense_layer_bias")
w = Variable(rand(Float64, (NUM_OF_CLASSES, 12 * 12)) ./ 10, name="dense_layer_weights")
w_conv = Variable(rand(Float64, 1, 9), name="convolution_weights")
w_conv2 = Variable(rand(Float64, 1, 9), name="convolution2_weights")
train_dataset = MNIST(:train)
N = size(train_dataset.features)[3]

60000

In [4]:
# Variables that will me modified on each run
img = Variable(Float64.(train_dataset[1].features), name="img")
actual_class = Variable(train_dataset[1].targets + 1, name="actual_class")
# Layers
conv_layer = conv(img, w_conv, Constant(3), Constant(3),Constant(1))
conv_layer2 = conv(conv_layer, w_conv2, Constant(3), Constant(3),Constant(1))
max_pool_layer = maxpool(conv_layer2, Constant(2))
flatten_layer = flatten(max_pool_layer)
fc_layer = dense(w, b, flatten_layer, softmax)
loss = crossentropy(fc_layer, actual_class)
net = topological_sort(loss)

19-element Vector{Any}:
 var dense_layer_weights
 ┣━ ^ 10×144 Matrix{Float64}
 ┗━ ∇ Nothing
 var img
 ┣━ ^ 28×28 Matrix{Float64}
 ┗━ ∇ Nothing
 var convolution_weights
 ┣━ ^ 1×9 Matrix{Float64}
 ┗━ ∇ Nothing
 const 3
 const 1
 op.?(typeof(conv))
 var convolution2_weights
 ┣━ ^ 1×9 Matrix{Float64}
 ┗━ ∇ Nothing
 op.?(typeof(conv))
 const 2
 op.?(typeof(maxpool))
 op.?(typeof(flatten))
 op.?(typeof(mul!))
 var dense_layer_bias
 ┣━ ^ 10-element Vector{Float64}
 ┗━ ∇ Nothing
 op.?(typeof(+))
 op.?(typeof(softmax))
 op.?(typeof(log))
 op.?(typeof(-))
 var actual_class
 ┣━ ^ Int64
 ┗━ ∇ Nothing
 op.?(typeof(select))

# Training network

In [5]:
losses = zeros(N)
for j = 1:EPOCHS
    for i = 1:N
        img.output = Float64.(train_dataset[i].features)
        actual_class.output = train_dataset[i].targets + 1
        loss_value = forward!(net)
        losses[i] = loss_value
        backward!(net)
        update_var!(b, LEARNING_RATE)
        update_var!(w, LEARNING_RATE)
        update_var!(w_conv2, LEARNING_RATE)
        update_var!(w_conv, LEARNING_RATE)
    end
    @printf("Avarage loss during epoch #%d run : %f \n", j, mean(losses))
end

Avarage loss during epoch #1 run : 0.567660 


Avarage loss during epoch #2 run : 0.497036 


Avarage loss during epoch #3 run : 0.466534 


# Testing network

In [7]:
test_dataset = MNIST(:test)
N = size(test_dataset.features)[3]
net = topological_sort(fc_layer)

@printf("Testing network...\n")
let count = 0
    for i = 1:N
        img.output = Float64.(test_dataset[i].features)
        y = forward!(net)
        if argmax(y) == (test_dataset[i].targets + 1)
            count += 1
        end
    end

    @printf("Network accuracy:  %f \n", count / N)
end

Testing network...


Network accuracy:  0.873600 
